In [1]:
import os
import sys
import glob

sys.path.append("/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D")

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from datetime import datetime

from utilities import fuse_landmarks_from_two_cameras
from functools import partial

import open3d as o3d
from utilities import convert_to_shoulder_coord
import time

from dataloader_transformer_encoder import HandArmLandmarksDataset_Transformer_Encoder
from transformer_encoder import TransformerEncoder


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
input_dim = 322
output_dim = 48 * 3  # 144
num_encoder_heads = 7
num_encoder_layers = 2
dim_feedforward = 164
dropout = 0.2
model = TransformerEncoder(input_dim=input_dim,
    output_dim=output_dim,
    num_heads=num_encoder_heads,
    num_encoder_layers=num_encoder_layers,
    dim_feedforward=dim_feedforward,
    dropout=dropout)

/home/giakhang/.local/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [3]:
DATA_DIR = "/home/giakhang/dev/pose_sandbox/data"  
SELECTED_DATE = "*"
train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]

sequence_length = 5
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200

train_dataset = HandArmLandmarksDataset_Transformer_Encoder(train_paths, 
    sequence_length,
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True)
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)

In [4]:
for inputs, targets in train_dataloader:
    inputs = inputs.permute(1, 0, 2)  # shape: [seq_length, B, input_dim]
    outputs = model(inputs)
    print(outputs.shape)
    break

torch.Size([512, 144])


In [6]:
positional_embedding = model._generate_positional_encoding(5, 322)

In [7]:
positional_embedding.shape

torch.Size([1, 5, 322])

In [23]:
pos = positional_embedding.repeat(512, 1, 1)
pos = pos.permute(1, 0, 2)

In [24]:
pos.shape

torch.Size([5, 512, 322])

In [26]:
np.sum((pos[:, 0, :] - pos[:, 1, :]).numpy())

0.0

In [8]:
model.cuda()

TransformerEncoder(
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=322, out_features=322, bias=True)
    )
    (linear1): Linear(in_features=322, out_features=164, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (linear2): Linear(in_features=164, out_features=322, bias=True)
    (norm1): LayerNorm((322,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((322,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.2, inplace=False)
    (dropout2): Dropout(p=0.2, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=322, out_features=322, bias=True)
        )
        (linear1): Linear(in_features=322, out_features=164, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (line